In [ ]:
!wget https://storage.googleapis.com/example-dataset-bucket/example-dataset.zip

In [ ]:
import numpy as np
import pandas as pd

import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# visualization
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import rasterio
from joblib import Parallel, delayed

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pydicom as dicom
import nibabel as nib
import sys
import glob
import os
import numpy as np
import h5py
from sklearn.preprocessing import StandardScaler

In [ ]:
import pydicom as dicom
import nibabel as nib
import sys
import glob
import os
import numpy as np
import h5py
from sklearn.preprocessing import StandardScaler

In [ ]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    array   = np.rot90(np.array(array),3)
    return(array)

In [ ]:
img_dir = r'C:\Users\User\Desktop\NIR\example-dataset\example-dataset'
dataset_X = []
dataset_y = []
for split in ['train','val']:
        files = glob.glob(os.path.join(img_dir, split, '**'))
        negative_samples = []
        for i, subject in enumerate(files):
            print('{}/{} {}'.format(i+1,len(files),subject))
            mask = os.path.join(subject, 'mask.nii')
            cor_t1_c = os.path.join(subject, 'COR_T1_C.nii')
            # cor_t1 = os.path.join(subject, 'COR_T1.nii')
            cor_t2 = os.path.join(subject, 'COR_T2.nii')

            sample_t1_c = read_nii(cor_t1_c)
            # sample_t1 = read_nii(cor_t1)
            sample_t2 = read_nii(cor_t2)
            sample_mask  = read_nii(mask)

            for i in range(sample_mask.shape[-1]):
                for j in [ sample_t1_c,sample_t2]:
                    if (sum(sum(sample_mask[...,i])))>0:
                            mask = sample_mask[...,i]
                            mask[mask==3] = 0
                            mask[mask==2] = 0
                            dataset_X.append(j[...,i])
                            dataset_y.append(sample_mask[...,i])




In [ ]:
img_dir = r'C:\Users\User\Desktop\NIR\example-dataset\example-dataset'
dataset_X = []
dataset_y = []
for split in ['train']:
        files = glob.glob(os.path.join(img_dir, split, '**'))
        negative_samples = []
        for i, subject in enumerate(files):
            print('{}/{} {}'.format(i+1,len(files),subject))
            mask = os.path.join(subject, 'mask.nii')
            cor_t1_c = os.path.join(subject, 'COR_T1_C.nii')
            cor_t1 = os.path.join(subject, 'COR_T1.nii')
            cor_t2 = os.path.join(subject, 'COR_T2.nii')
            
            sample_t1_c = read_nii(cor_t1_c)
            sample_t1 = read_nii(cor_t1)
            sample_t2 = read_nii(cor_t2)
            sample_mask  = read_nii(mask)

            for i in range(sample_mask.shape[-1]):
                for j in [ sample_t1_c,sample_t1,sample_t2]:
                    if (sum(sum(sample_mask[...,i])))>0:
                            mask = sample_mask[...,i]
                            mask[mask==3] = 0
                            mask[mask==2] = 0
                            dataset_X.append(j[...,i])
                            dataset_y.append(sample_mask[...,i])

dataset_y= np.expand_dims(dataset_y, axis = -1)

In [ ]:
np.expand_dims(dataset_X, axis = -1).shape

In [ ]:
img_dir = r'C:\Users\User\Desktop\NIR\example-dataset\example-dataset'
dataset_X_v = []
dataset_y_v = []
for split in ['val']:
        files = glob.glob(os.path.join(img_dir, split, '**'))
        negative_samples = []
        for i, subject in enumerate(files):
            print('{}/{} {}'.format(i+1,len(files),subject))
            mask = os.path.join(subject, 'mask.nii')
            cor_t1_c = os.path.join(subject, 'COR_T1_C.nii')
            # cor_t1 = os.path.join(subject, 'COR_T1.nii')
            # cor_t2 = os.path.join(subject, 'COR_T2.nii')

            sample_t1_c = read_nii(cor_t1_c)
            # sample_t1 = read_nii(cor_t1)
            # sample_t2 = read_nii(cor_t2)
            sample_mask  = read_nii(mask)

            for i in range(sample_mask.shape[-1]):
                for j in [ sample_t1_c]:
                    if (sum(sum(sample_mask[...,i])))>0:
                            mask = sample_mask[...,i]
                            mask[mask==3] = 0
                            mask[mask==2] = 0
                            dataset_X_v.append(j[...,i])
                            dataset_y_v.append(sample_mask[...,i])
dataset_y_v= np.expand_dims(dataset_y_v, axis = -1)

In [ ]:
def read_labels_from_file(file_path):
    values = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            try:
                value = int(line.split(': ')[1])
            except ValueError:
                value = line.split(': ')[1].strip()  # Если значение не может быть преобразовано в int, оставляем его как строку
            if str(value) in ['3а'  ,'3','3a']:
                value = '3a'
            values.append(value)
    return values

In [ ]:
file_path = os.path.join('data_TEST/ID_65', 'labels.txt')

values = read_labels_from_file('data_TEST/ID_89/labels.txt')
print(values)

In [ ]:
def load_tif(x_path, y_path,count_dict, step=4):
    images = []
    masks = []
    mean = []
    image = Image.open(x_path)
    mask = Image.open(y_path)
    i = 0
    cnt = 0
    
    while True:
        try:
            mask.seek(i)
            mask_array = np.array(mask)
            image.seek(i)
            image_array = np.array(image)
            mean.append(sum(sum(mask_array)))
            if sum(sum(mask_array)) > 0: #Проверка, что маска не полностью черная
                    masks.append(mask_array)
                    images.append(image_array)
            cnt+=1
                    
            i += 1
        except EOFError:
            break

    if 'T1' in x_path:
        count_dict['scan'].append('T1')
    elif 'T2' in x_path:
        count_dict['scan'].append('T2')     
    count_dict['count'].append(cnt)
    
    return images, masks

In [ ]:
def get_images(x_pathes, y_pathes):

    images =[]
    masks = []
    count_dict = {'scan':[],'count':[],}
    for x_path, y_path in zip(x_pathes, y_pathes):
          
          images_sample,mask_sample = load_tif(x_path, y_path,count_dict)
          for im,ms in zip(images_sample,mask_sample):
                
                images.append(im)
                masks.append(ms)
            #     fig , ax =  plt.subplots( 1,2, figsize=(18, 18))
            #     # for i in range(len(ms[7:14])):
            #     # ax[i][0].imshow(im[i])
            #     # ax[i][1].imshow(ms[i])
            #     ax[0].imshow(im)
            #     ax[1].imshow(im)
            #     ax[1].imshow(ms, alpha=0.5)
            #     plt.show()
    print(len(count_dict['count']))
    return images, masks ,count_dict

In [ ]:
def get_images_mean(x_pathes, y_pathes):

    images =[]
    masks = []
    count_dict = {'scan':[],'count':[],}
    for x_path, y_path in zip(x_pathes, y_pathes):
          
          images_sample,mask_sample = load_tif(x_path, y_path,count_dict)
          for im,ms in zip(images_sample,mask_sample):
                
                images.append(im)
                masks.append(ms)
            #     fig , ax =  plt.subplots( 1,2, figsize=(18, 18))
            #     # for i in range(len(ms[7:14])):
            #     # ax[i][0].imshow(im[i])
            #     # ax[i][1].imshow(ms[i])
            #     ax[0].imshow(im)
            #     ax[1].imshow(im)
            #     ax[1].imshow(ms, alpha=0.5)
            #     plt.show()
    print(len(count_dict['count']))
    return images, masks ,count_dict

In [ ]:
def get_pathes(path):
    x_pathes_all = []
    y_pathes_all = []
    for patient in os.listdir(path):
        x_pathes = []
        y_pathes = []
      
        for ID_s in os.listdir(path + '/'+ patient ):
            if 'ID' in ID_s:
                msk_t1 = 0
                msk_t2 = 0  
                for tif_name in os.listdir(path + '/'+ patient + '/'+ID_s):
    
                    if 'Cor' in tif_name:
                        if 'T1' in tif_name:
            
                            if  'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/' + ID_s + '/'+ tif_name)

                            elif 'mask' in tif_name.lower():
                                msk_t1 = 1
                                y_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)

                        elif 'T2' in tif_name:
                            
                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)
                              
                            elif 'mask' in tif_name.lower():
                                msk_t2 = 1
                                y_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)
                        
                if msk_t1==0:
                    x_pathes.pop()
                if msk_t2==0:
                    x_pathes.pop()
                        
        x_pathes_all.append(x_pathes)
        y_pathes_all.append(y_pathes)
                
                
    return x_pathes_all, y_pathes_all


In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [ ]:
x_pth ,y_pth = get_pathes('data_08_06_24_cls141_seg105')

In [ ]:
x_pth

In [ ]:
len(im)

In [ ]:
x_pth ,y_pth = [flatten(i) for i in [x_pth ,y_pth]]

In [ ]:
im,ms,cnt= get_images(x_pth ,y_pth)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
max_length = image_count_all['count'].apply(len).max()
image_count_all['count'] = image_count_all['count'].apply(lambda x: x + [0]*(max_length-len(x)))

# Вычисляем средние значения пикселей по каждой позиции для каждого типа сканирования
averages = image_count_all.groupby('scan')['count'].apply(lambda x: np.mean(x.tolist(), axis=0)).reset_index()

# Построение графиков с наложением
plt.figure(figsize=(10, 6))

# Проходимся по всем строкам DataFrame и строим графики
for index, row in averages.iterrows():
    plt.plot(row['count'], label=row['scan'])

# Добавляем легенду и отображаем график
plt.legend()
plt.xlabel('Position')
plt.ylabel('Average Pixel Value')
plt.title('Average Pixel Distribution by Position for Each Scan')

In [ ]:
image_count_all= pd.DataFrame(cnt)
image_count_all

In [ ]:
# image_count_all= pd.DataFrame(cnt,columns=['type','count'])
sns.histplot(data = image_count_all,x='count',hue='scan', stat="density")

In [ ]:
# image_count_all= pd.DataFrame(cnt,columns=['image_count_all'])
sns.histplot(data = image_count_all,x='count')
    stat="density")

In [ ]:
len(im)

In [ ]:
len(ms)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_pth ,y_pth = [flatten(i) for i in [x_pth ,y_pth]]

In [ ]:
y_pth

In [ ]:
x_pth

In [ ]:
im,ms,cnt = get_images(x_pth ,y_pth)

In [ ]:
import seaborn as sns

In [ ]:
image_count_all= pd.DataFrame(cnt,columns=['image_count_all'])

In [ ]:
image_count_all

In [ ]:
sns.histplot(data = image_count_all,x='image_count_all',
    stat="density")

In [ ]:
sns.histplot(data = image_count,x='image_count',
    stat="density")

In [ ]:
len(ms)

In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [ ]:
x_pth_train,x_pth_val,y_pth_train,y_pth_val = train_test_split(x_pth , y_pth, test_size=0.14)

print(x_pth_train)
x_pth_train,y_pth_train,x_pth_val,y_pth_val = flatten(x_pth_train) ,flatten(y_pth_train),flatten(x_pth_val),flatten(y_pth_val)

In [ ]:
len(y_pth_val)

In [ ]:
len(y_pth_val)

In [ ]:
x_pth_train

In [ ]:
y_pth_train

In [ ]:
y_pth_train

In [ ]:
x_pth_train

In [ ]:
x_train,y_train,_ = get_images(x_pth_train ,y_pth_train)

In [ ]:
x_val,y_val,_ = get_images(x_pth_val ,y_pth_val) 
x_train,y_train,_= get_images(x_pth_train ,y_pth_train)

In [ ]:
x_pth_val


In [ ]:
x_val,y_val,_ = get_images(x_pth_val ,y_pth_val) 

In [ ]:
len(x_train)

In [ ]:
len(y_train)

In [ ]:
class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, X,y, label=True, transforms=None):
        self.label      = label
        self.img_paths  = X
        self.msk_paths  = y
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        if self.label == True:
            img = self.img_paths[index]
            msk = self.msk_paths[index]
            if self.transforms:
                data = self.transforms(image=np.array(img/255., dtype=np.float32)  , mask=np.array(msk, dtype=np.float32))
                img  = data['image']
                msk  = data['mask']

            return torch.tensor([img]), torch.tensor([msk])
        else:
            img = self.img_paths[index]
            if self.transforms:
                data = self.transforms(image=np.array(img/255., dtype=np.float32))
                img  = data['image']
            return torch.tensor([img])

In [ ]:
# class BuildDataset(torch.utils.data.Dataset):
#     def __init__(self, X,y, label=True, transforms=None):
   
#         self.label      = label
#         self.img_paths  = X
#         self.msk_paths  = y
#         self.transforms = transforms
        
#     def __len__(self):
#         return len(self.img_paths)
    
#     def __getitem__(self, index):
#         # img_path  = self.img_paths[index]
#         # img = []
#         # img = load_img(img_path)
#         img = self.img_paths[index]
        
#         if self.label:
#             msk = self.msk_paths[index]
            
#             if self.transforms:
#                 data = self.transforms(image=np.array(img/255., dtype=np.float32)  , mask=np.array(msk, dtype=np.float32))
#                 img  = data['image']
#                 msk  = data['mask']
#                 print(sum(msk))
#             msk = np.transpose(msk, (2, 0, 1))
#             return torch.tensor([img]), torch.tensor(msk)
#         else:
#             if self.transforms:
#                 data = self.transforms(image=np.array(img/255., dtype=np.float32))
#                 img  = data['image']
#             return torch.tensor([img])

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(224,224),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=1.),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
# #             A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.25),
        A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=1.0),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ]
        , p=1.0),
    
    "valid": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(224,224),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}

In [ ]:
train_dataset = BuildDataset( np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0), transforms=data_transforms['valid'])
# train_dataset = BuildDataset(  x_train, y_train, transforms=None)
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_loader = DataLoader(train_dataset, batch_size=5,shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True)


In [ ]:
md = SegmentationModel(model_type='seg_test', base_dir="" )
res = md.predict(x_train[0:15])

In [ ]:
plt.imshow(res[14][0])

In [ ]:
res

In [ ]:
plt.imshow(md.result_masks[10])

In [ ]:
import cv2 as cv
for k,seg_def in enumerate(res):
       
        
        c,h = cv.findContours(seg_def[0],cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE | cv.CHAIN_APPROX_TC89_L1)
        
        if h is not None:
            counter = [c[i] for i in range(h.shape[1]) if h[0][i][3] == -1][0]
            for i, pi in enumerate(counter):
                print(
                    i,
                    pi[0,0],
                    pi[0,1],
                    seg_def[2]
                )


In [ ]:
for k,seg_def in enumerate(res):    
    # if seg_def[0].dtype != np.uint8:
    #     seg_def[0] = seg_def[0].astype(np.uint32)
    print(seg_def[1])
    # Удостоверимся, что изображение бинарное
    _, binary_image = cv.threshold(seg_def[0], 127, 255, cv.THRESH_BINARY)
    
    contours, hierarchy = cv.findContours(binary_image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    for i, contour in enumerate(contours):
        print(f"Contour #{i}: {contour}")

In [ ]:
hierarchy

In [ ]:
for nd in res:
    print(nd[0],nd[0])
    break

In [ ]:
md.predict(x_train)

In [ ]:
i = 0
for imgs, msks in  iter(train_dataset):
    i+=1
print(i)


In [ ]:
fig , ax =  plt.subplots( 1,2, figsize=(18, 18))
# for i in range(len(ms[7:14])):
# ax[i][0].imshow(im[i])
# ax[i][1].imshow(ms[i])
ax[0].imshow(x_train[135])
ax[1].imshow(x_train[135])
ax[1].imshow(y_train[135], alpha=0.5)
plt.show()

In [ ]:
plt.imshow(x_train[135])

In [ ]:
imgs, msks = next(iter(train_loader))
imgs.size(), msks.size()

In [ ]:
def plot_batch(imgs, msks, size=3):
    plt.figure(figsize=(5*5, 5))
    for idx in range(size):
        plt.subplot(1, size, idx+1)
        img = imgs[idx,].permute((1, 2, 0)).numpy()*255.0
 
        msk = msks[idx,].permute((1, 2, 0)).numpy()
        show_img(img, msk)
    plt.tight_layout()
    plt.show()

In [ ]:
def show_img(img, mask=None):

#     img = clahe.apply(img)
#     plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='bone')
    
    
        # plt.imshow(np.ma.masked_where(mask!=1, mask), alpha=0.5, cmap='autumn')
    plt.imshow(mask, alpha=0.5)
    handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0)]]
    labels = ["Adenoma"]
    plt.legend(handles,labels)
    plt.axis('off')

In [ ]:
msks.shape

In [ ]:
train_dataset = BuildDataset( np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0), transforms=None)
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [ ]:
len(np.concatenate((dataset_X, x_train)))

In [ ]:
y_train

In [ ]:

imgs, msks = next(iter(train_loader))
imgs.size(), msks.size()
plot_batch(imgs, msks, size=4)

In [ ]:
plot_batch(imgs, imgs, size=5)

In [ ]:
imgs, msks = next(iter(train_loader))
imgs.size(), msks.size()
plot_batch(imgs, msks, size=5)

In [ ]:
plot_batch(imgs, imgs, size=5)

In [ ]:
plot_batch(msks, msks, size=5)

In [ ]:
import gc
gc.collect()

In [ ]:
import segmentation_models_pytorch as smp

def build_model():
    model = smp.UnetPlusPlus(
        encoder_name='efficientnet-b1' ,     # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights='imagenet',     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1   ,     # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to('cuda')
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [ ]:
model

In [ ]:
JaccardLoss = smp.losses.JaccardLoss(mode='binary')
DiceLoss    = smp.losses.DiceLoss(mode='binary')
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
LovaszLoss  = smp.losses.LovaszLoss(mode='binary', per_image=False)
TverskyLoss = smp.losses.TverskyLoss(mode='binary', log_loss=False)

def dice_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

def iou_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    union = (y_true + y_pred - y_true*y_pred).sum(dim=dim)
    iou = ((inter+epsilon)/(union+epsilon)).mean(dim=(1,0))
    return iou

def criterion(y_pred, y_true):
    return BCELoss(y_pred, y_true) +TverskyLoss(y_pred, y_true)
    # return 0.5*BCELoss(y_pred, y_true) + 0.5*TverskyLoss(y_pred, y_true)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    train_scores = []
    dataset_size = 0
    running_loss = 0.0
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, masks) in pbar:         
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            
            loss   = criterion(y_pred, masks)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size

        y_pred = (nn.Sigmoid()(y_pred)).double()
        train_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        vtrain_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        train_scores.append([train_dice, vtrain_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
        
    train_scores  = np.mean(train_scores, axis=0)    
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss,train_scores

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (images, masks) in pbar:        
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        loss    = criterion(y_pred, masks)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        y_pred = (nn.Sigmoid()(y_pred)).double()
        val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
        
        
    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores

In [ ]:
time.time()

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    # wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_dice      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss,train_scores = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        train_dice,train_jaccard = train_scores

        print(f'Train Dice: {train_dice:0.4f} | Train Jaccard: {train_jaccard:0.4f}')

        val_loss, val_scores = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch)
        val_dice, val_jaccard = val_scores
    
        history['Train Loss'].append(train_loss)
        history['Train Dice'].append(train_dice)
        history['Train Jaccard'].append(train_jaccard)
        history['Valid Loss'].append(val_loss)
        history['Valid Dice'].append(val_dice)
        history['Valid Jaccard'].append(val_jaccard)
        
        # Log the metrics
        # wandb.log({"Train Loss": train_loss, 
        #            "Valid Loss": val_loss,
        #            "Valid Dice": val_dice,
        #            "Valid Jaccard": val_jaccard,
        #            "LR":scheduler.get_last_lr()[0]})
        
        print(f'Valid Dice: {val_dice:0.4f} | Valid Jaccard: {val_jaccard:0.4f}')
        
        # deep copy the model
        if val_dice > best_dice:
            print(f"{c_}Valid Score Improved ({best_dice:0.4f} ---> {val_dice:0.4f})")
            best_dice    = val_dice
            best_jaccard = val_jaccard
            best_epoch   = epoch
            run.summary["Best Dice"]    = best_dice
            run.summary["Best Jaccard"] = best_jaccard
            run.summary["Best Epoch"]   = best_epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            # wandb.save(PATH)
            print(f"Model Saved{sr_}")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
            
        print(); print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Score: {:.4f}".format(best_jaccard))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
class CFG:
    seed          = 42
    debug         = False # set debug=False for Full Training
    exp_name      = 'Baselinev2'
    comment       = 'unet-efficientnet_b1-224x224-aug2-split2'
    model_name    = 'Unet'
    backbone      = 'efficientnet-b1'
    train_bs      = 128
    valid_bs      = train_bs*2
    img_size      = [224, 224]
    epochs        = 200
    lr            = 1e-4
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 5
    num_classes   = 3
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
help(smp)

In [ ]:
def build_model():
    model = smp.DeepLabV3Plus(
        encoder_name='mobilenet_v2' ,     # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1   ,     # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to('cuda')
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [ ]:
model = build_model()
optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
scheduler = fetch_scheduler(optimizer)

In [ ]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None
        
    return scheduler

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(256,256, interpolation=cv2.INTER_NEAREST),
        A.CoarseDropout(max_holes=8, max_height=256//20, max_width=256//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        

        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
            # A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.25),
        # A.ColorJitter(brightness=0, contrast=0.0002, saturation=0, hue=0.2, always_apply=False, p=0.5),
        # A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
        #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0),
    
    "valid": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(256,256, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}
train_dataset = BuildDataset(np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0), transforms=data_transforms['train'])
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_loader = DataLoader(train_dataset, batch_size=32,shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
# train_dataset = BuildDataset(dataset_X,dataset_y, transforms=data_transforms['train'])
# valid_dataset = BuildDataset(dataset_X_v,dataset_y_v, transforms=data_transforms['valid'])

# train_loader = DataLoader(train_dataset, batch_size=16,shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:
# np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0)

train_dataset = BuildDataset(np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0), transforms=data_transforms['train'])
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_loader = DataLoader(train_dataset, batch_size=32,shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
len(valid_dataset)

In [ ]:
train_dataset_1 = BuildDataset(x_train,y_train, transforms=data_transforms['train'])
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_loader = DataLoader(train_dataset, batch_size=64,shuffle=True, pin_memory=True, drop_last=False)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [ ]:
data_transforms_v= {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(256,256, interpolation=cv2.INTER_NEAREST),
        A.CoarseDropout(max_holes=8, max_height=256//20, max_width=256//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        

        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
            # A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.25),
        # A.ColorJitter(brightness=0, contrast=0.0002, saturation=0, hue=0.2, always_apply=False, p=0.5),
        # A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
        #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0),
    
    "valid": A.Compose([
        # A.augmentations.crops.transforms.CenterCrop(256,256),
        # A.Resize(256,256, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}
valid_dataset_v = BuildDataset(x_val,y_val, transforms=data_transforms_v['valid'])

valid_loader_v = DataLoader(valid_dataset_v, batch_size=4, shuffle=False, pin_memory=True)

In [ ]:
imgs,msks = next(iter(valid_loader_v))
msks.size()

In [ ]:
from typing import Iterable

In [ ]:
def center_crop(img, output_size):
    _, _, h, w = img.size()
    new_w, new_h = output_size

    start_x = (w - new_w)//2
    start_y = (h - new_h)//2

    return img[:, :, start_y:start_y + new_h, start_x:start_x + new_w]

def crop_and_segment(image, orig_size=(512, 512), crop_size=(224, 224)):
    
    # Центральное обрежем изображение
    cropped_image = center_crop(image, crop_size)

    # Отмасштабируем обратно к исходному размеру
    mask_upscaled = F.resize(cropped_image, orig_size, interpolation=F.InterpolationMode.NEAREST)

    return mask_upscaled

In [ ]:
def crop_and_segment(image, orig_size=(512, 512), crop_size=(224, 224)):
    
    # Центральное обрежем изображение
    cropped_image = center_crop(image, crop_size)

    # Отмасштабируем обратно к исходному размеру
    mask_upscaled = F.resize(cropped_image, orig_size, interpolation=F.InterpolationMode.NEAREST)

    return mask_upscaled

In [ ]:
def center_crop(img, output_size):
    _, h, w = img.size()
    new_w, new_h = output_size

    start_x = (w - new_w)//2
    start_y = (h - new_h)//2

    return img[ :, start_y:start_y + new_h, start_x:start_x + new_w]
crop_size=(224, 224)
orig_size=(512, 512)
print(type(crop_size),crop_size)
cropped_image = center_crop(msks, crop_size)
padded_imgs = T.Pad(padding=((512 - 224)//2))(cropped_image) 
print(padded_imgs.size())
aaa =  padded_imgs

In [ ]:
# msks_2 = crop_and_segment(msks)
crop_size=(224, 224)
orig_size=(512, 512)

cropped_image = center_crop(msks, crop_size)
padded_imgs = T.Pad(padding=((512 - 224)//2))(cropped_image) 
print(padded_imgs.size())
aaa =  padded_imgs

In [ ]:
type(cropped_image)

In [ ]:
padded_imgs = T.Pad(padding=((512 - 224)//2))(cropped_image) 
print(padded_imgs.size())
aaa =  padded_imgs
plot_batch(aaa, aaa, size=2)
plot_batch(cropped_image, cropped_image, size=2)


In [ ]:
plot_batch(msks_2, msks_2, size=2)
plot_batch(cropped_image, cropped_image, size=2)
plot_batch(msks, msks, size=2)

In [ ]:
mask_upscaled = F.resize(cropped_image, orig_size, interpolation=F.InterpolationMode.NEAREST)

In [ ]:
padded_imgs = T.Pad(padding=((512 - 224)//2))(cropped_image) 
print(padded_imgs.size())
aaa = cropped_image + padded_imgs
plot_batch(cropped_image, cropped_image, size=2)
plot_batch(aaa, aaa, size=2)
plot_batch(imgs, imgs, size=2)
plot_batch(imgs, aaa, size=2)

In [ ]:
from torchvision.transforms import functional as F
from skimage.transform import resize
# Assume that mask_224 is our mask to be resized
mask_resized = F.resize(msks, [512, 512], interpolation=F.InterpolationMode.BILINEAR)


In [ ]:
t = resize(msks[0], (512, 512), mode='constant', anti_aliasing=True,)

In [ ]:
from skimage.transform import resize
size = (512, 512)
X = [resize(x, size, mode='constant', anti_aliasing=True,) for x in dataset_X]
Y = [resize(y, size, mode='constant', anti_aliasing=False) > 0.5 for y in dataset_y]

In [ ]:
from torchvision import transforms as T

In [ ]:
preprocessing(msks[0])[0].shape

In [ ]:
plt.imshow(pt)

In [ ]:
imgs,msks = next(iter(valid_loader_v))
plot_batch(imgs, msks, size=5)
plot_batch(imgs, imgs, size=5)
plot_batch(msks_2, msks_2, size=1)

In [ ]:
imgs,msks = next(iter(valid_loader))
plot_batch(imgs, msks, size=5)
plot_batch(imgs, imgs, size=5)

In [ ]:
len(train_dataset)

In [ ]:
len(valid_dataset)

In [ ]:
history_all_sota

In [ ]:
history_all_sota = defaultdict(list)
encoder_list = [
    # 'efficientnet-b0',
    # 'efficientnet-b1',
    # 'efficientnet-b2',
    # 'efficientnet-b3',
    # 'efficientnet-b4',
    # 'efficientnet-b5',
    # 'efficientnet-b6',
    # 'efficientnet-b7',
    # "resnet18",
    # "resnet34",
    # "resnet50",
    # "resnet101",
    # "resnet152",
    # "resnext50_32x4d",
    # # "resnext101_32x8d",
    # # "resnext101_32x16d",
    # "mobilenet_v2",
    # "timm-mobilenetv3_large_075",
    # "timm-mobilenetv3_large_100",
    # "timm-mobilenetv3_large_minimal_100",
    # "mit_b0",
    # "mit_b1",
    # "mit_b2",
    # "mit_b3",
    # "mit_b4",
    # "mit_b5",
"efficientnetv2_l",
"efficientnetv2_m",
"efficientnetv2_rw_m",
"efficientnetv2_rw_s",
"efficientnetv2_rw_t",
"efficientnetv2_s",
"efficientnetv2_xl",
    ]

# encoder_name_list = [str(string).split(".")[-1][:-2] for string in model_list]
for seg_model,seg_model_name in zip(encoder_list,encoder_list):
    def build_model():
        model = smp.UnetPlusPlus(
            encoder_name='tu-'+seg_model ,     # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights=None,# use `imagenet` pre-trained weights for encoder initialization
            in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=1   ,     # model output channels (number of classes in your dataset)
            activation=None,
        )
        model.to('cuda')
        return model

    def load_model(path):
        model = build_model()
        model.load_state_dict(torch.load(path))
        model.eval()
        return model

    for fold in range(1):
        print(f'#'*15)
        print(f'### Fold: {fold}')
        print(f'#'*15)
        # run = wandb.init(project='uw-maddison-gi-tract', 
        #                  config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
        #                  anonymous=anonymous,
        #                  name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
        #                  group=CFG.comment,
        #                 )
        train_loader, valid_loader = train_loader,valid_loader
        model     =   build_model()
        # UNet(1,1).to('cuda')
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
        scheduler =  fetch_scheduler(optimizer) 
        # fetch_scheduler(optimizer)
        
        model, history = run_training(model, optimizer, scheduler,
                                    device=CFG.device,
                                    num_epochs=CFG.epochs)
        # run.finish()
        # display(ipd.IFrame(run.url, width=1000, height=720))
        plt.figure(figsize=(12,9))

        plt.plot(history['Train Loss'], label=f'Training Loss')
        plt.plot(history['Valid Loss'], label=f'Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title(f'Loss History-model{seg_model_name}')


        plt.figure(figsize=(17,9))
        plt.subplot(1, 2, 1)
        plt.plot(history['Train Dice'], label=f'Training Dice')
        plt.plot(history['Valid Dice'], label=f'Validation Dice')
        plt.xlabel('Epoch')
        plt.ylabel('Dice')
        plt.legend()
        plt.title(f'Dice History-model{seg_model_name}')

        plt.subplot(1, 2, 2)
        plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
        plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
        plt.xlabel('Epoch')
        plt.ylabel('Jaccard')
        plt.legend()
        plt.title(f'Jaccard History-model{seg_model_name}')
        plt.show()

    history_all_sota[f'{seg_model_name}'].append(history)
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    GPUtil.showUtilization()
    free_memory = gpu.memoryFree
    torch.cuda.empty_cache()

for seg_model_name, history in history_all_sota.items():

        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
    plt.figure(figsize=(12,9))

    plt.plot(history['Train Loss'], label=f'Training Loss')
    plt.plot(history['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(history['Train Dice'], label=f'Training Dice')
    plt.plot(history['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()

In [ ]:
    plt.figure(figsize=(12,9))

    plt.plot(history['Train Loss'], label=f'Training Loss')
    plt.plot(history['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(history['Train Dice'], label=f'Training Dice')
    plt.plot(history['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()

In [ ]:
max(history['Valid Jaccard'])

In [ ]:
    plt.figure(figsize=(12,9))

    plt.plot(historys['Train Loss'], label=f'Training Loss')
    plt.plot(historys['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(historys['Train Dice'], label=f'Training Dice')
    plt.plot(historys['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(historys['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(historys['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()

In [ ]:
def build_model():
    model = smp.UnetPlusPlus(
        encoder_name='efficientnet-b6' ,
                  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1   ,     # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to('cuda')
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

for fold in range(1):
    seg_model_name = 'UNetPPEffb6_LAST'
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    run = wandb.init(project='uw-maddison-gi-tract', 
                     config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                     anonymous=anonymous,
                     name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
                     group=CFG.comment,
                    )
    train_loader, valid_loader = train_loader,valid_loader
    model     =   build_model()
    # UNet(1,1).to('cuda')
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler =  fetch_scheduler(optimizer) 
    # fetch_scheduler(optimizer)
    
    model, historys_UNET = run_training(model, optimizer, scheduler,
                                device=CFG.device,
                                num_epochs=CFG.epochs)
    # run.finish()
    # display(ipd.IFrame(run.url, width=1000, height=720))
    plt.figure(figsize=(12,9))

    plt.plot(historys_UNET['Train Loss'], label=f'Training Loss')
    plt.plot(historys_UNET['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(historys_UNET['Train Dice'], label=f'Training Dice')
    plt.plot(historys_UNET['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(historys_UNET['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(historys_UNET['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()
    torch.save(model.state_dict(), f'{seg_model_name}_std.pth')
    torch.save(model, f'{seg_model_name}.pt')

In [ ]:
history

In [ ]:
    torch.save(model, 'UNetPPEffb6_600_2.pt')


In [ ]:
plt.figure(figsize=(12,9))

plt.plot(historys_UNET['Train Loss'], label=f'Training Loss')
plt.plot(historys_UNET['Valid Loss'], label=f'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f'Loss History-model{seg_model_name}')


plt.figure(figsize=(17,9))
plt.subplot(1, 2, 1)
plt.plot(historys_UNET['Train Dice'], label=f'Training Dice')
plt.plot(historys_UNET['Valid Dice'], label=f'Validation Dice')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()
plt.title(f'Dice History-model{seg_model_name}')

plt.subplot(1, 2, 2)
plt.plot(historys_UNET['Train Jaccard'], label=f'Training Jaccard')
plt.plot(historys_UNET['Valid Jaccard'], label=f'Validation Jaccard')
plt.xlabel('Epoch')
plt.ylabel('Jaccard')
plt.legend()
plt.title(f'Jaccard History-model{seg_model_name}')
plt.show()

In [ ]:
    plt.figure(figsize=(12,9))

    plt.plot(history['Train Loss'], label=f'Training Loss')
    plt.plot(history['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(history['Train Dice'], label=f'Training Dice')
    plt.plot(history['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()

In [ ]:
def build_model():
    model = smp.FPN(
        encoder_name='efficientnet-b6' ,
                  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1   ,     # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to('cuda')
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

for fold in range(1):
    seg_model_name = 'UNetPPEffb6_600_3'
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    run = wandb.init(project='uw-maddison-gi-tract', 
                     config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                     anonymous=anonymous,
                     name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
                     group=CFG.comment,
                    )
    

    data_transforms = {
        "train": A.Compose([
            A.augmentations.crops.transforms.CenterCrop(224,224),
            A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
            A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
                            min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
            

            A.HorizontalFlip(p=0.5),
    #         A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
            A.OneOf([
                A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
                # A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
                A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
            ], p=0.25),
            # A.ColorJitter(brightness=0, contrast=0.0002, saturation=0, hue=0.2, always_apply=False, p=0.5),
            # A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
            #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
            A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
            ], p=1.0),
        
        "valid": A.Compose([
            A.augmentations.crops.transforms.CenterCrop(224,224),
            A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
            A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
            ], p=1.0)
    }
    train_dataset = BuildDataset(np.concatenate((dataset_X, x_train), axis=0),np.concatenate((dataset_y, y_train), axis=0), transforms=data_transforms['train'])
    valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

    train_loader = DataLoader(train_dataset, batch_size=64,shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
    train_loader, valid_loader = train_loader,valid_loader
    model     =   build_model()
    # UNet(1,1).to('cuda')
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler =  fetch_scheduler(optimizer) 
    # fetch_scheduler(optimizer)
    
    model, historys = run_training(model, optimizer, scheduler,
                                device=CFG.device,
                                num_epochs=CFG.epochs)
    # run.finish()
    # display(ipd.IFrame(run.url, width=1000, height=720))
    plt.figure(figsize=(12,9))

    plt.plot(historys['Train Loss'], label=f'Training Loss')
    plt.plot(historys['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(historys['Train Dice'], label=f'Training Dice')
    plt.plot(historys['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(historys['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(historys['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()
    torch.save(model.state_dict(), f'{seg_model_name}_std.pth')
    torch.save(model, f'{seg_model_name}.pt')

In [ ]:
plt.figure(figsize=(12,9))

plt.plot(historys['Train Loss'], label=f'Training Loss')
plt.plot(historys['Valid Loss'], label=f'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f'Loss History-model{seg_model_name}')


plt.figure(figsize=(17,9))
plt.subplot(1, 2, 1)
plt.plot(historys['Train Dice'], label=f'Training Dice')
plt.plot(historys['Valid Dice'], label=f'Validation Dice')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()
plt.title(f'Dice History-model{seg_model_name}')

plt.subplot(1, 2, 2)
plt.plot(historys['Train Jaccard'], label=f'Training Jaccard')
plt.plot(historys['Valid Jaccard'], label=f'Validation Jaccard')
plt.xlabel('Epoch')
plt.ylabel('Jaccard')
plt.legend()
plt.title(f'Jaccard History-model{seg_model_name}')
plt.show()

In [ ]:

    history_all_sota[f'{seg_model_name}'].append(history)
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    GPUtil.showUtilization()
    free_memory = gpu.memoryFree
    torch.cuda.empty_cache()

In [ ]:
torch.save(model, 'UNetPPEffb1_300_bs64.pt')

In [ ]:
torch.save(model.state_dict(), 'UNetPPEffb1_300_bs64_state_dict')

In [ ]:
run = wandb.init(project='uw-maddison-gi-tract', 
                     config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                     anonymous=anonymous,
                     name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
                     group=CFG.comment,
                    )

In [ ]:
print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(historys['Train Dice'])} IOU MAX:{max(historys['Train Jaccard'])} Train Loss MIN:{min(historys['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(historys['Valid Dice'])} VAL IOU MAX:{max(historys['Valid Jaccard'])} VAL Loss MIN:{min(historys['Valid Loss'])} \n """)

In [ ]:
print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(historys_UNET['Train Dice'])} IOU MAX:{max(historys_UNET['Train Jaccard'])} Train Loss MIN:{min(historys_UNET['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(historys_UNET['Valid Dice'])} VAL IOU MAX:{max(historys_UNET['Valid Jaccard'])} VAL Loss MIN:{min(historys_UNET['Valid Loss'])} \n """)

In [ ]:
history_all = defaultdict(list)
model_list = [
        smp.Unet,
        smp.UnetPlusPlus,
        smp.MAnet,
        smp.Linknet,
        smp.FPN,
        smp.PSPNet,
        smp.DeepLabV3,
        smp.DeepLabV3Plus,
        smp.PAN
    ]

desired_list = [str(string).split(".")[-1][:-2] for string in model_list]

In [ ]:
   "resnet18",
    "resnet34",
    "resnet50",
    "resnet101",
    "resnet152",
    "resnext50_32x4d",
    "resnext101_32x4d",
    "resnext101_32x8d",
    "resnext101_32x16d",
    "resnext101_32x32d",
    "resnext101_32x48d",

In [ ]:
import GPUtil
import torch

gpus = GPUtil.getGPUs()

gpu = gpus[0]

GPUtil.showUtilization()

free_memory = gpu.memoryFree

torch.cuda.empty_cache()


In [ ]:
gpus = GPUtil.getGPUs()

gpu = gpus[0]

GPUtil.showUtilization()

free_memory = gpu.memoryFree

torch.cuda.empty_cache()


In [ ]:
history_all_encoder_2 = defaultdict(list)
encoder_list = [
    # 'efficientnet-b0',
    # 'efficientnet-b1',
    # 'efficientnet-b2',
    # 'efficientnet-b3',
    # 'efficientnet-b4',
    # 'efficientnet-b5',
    # 'efficientnet-b6',
    # 'efficientnet-b7',
    # "resnet18",
    # "resnet34",
    # "resnet50",
    # "resnet101",
    # "resnet152",
    # "resnext50_32x4d",
    # # "resnext101_32x8d",
    # # "resnext101_32x16d",
    # "mobilenet_v2",
    # "timm-mobilenetv3_large_075",
    # "timm-mobilenetv3_large_100",
    # "timm-mobilenetv3_large_minimal_100",
    # "mit_b0",
    # "mit_b1",
    # "mit_b2",
    # "mit_b3",
    # "mit_b4",
    # "mit_b5",
    "mobileone_s4",
    "mobileone_s3",
    "mobileone_s2",
    "mobileone_s1",
    "vgg11",
    "vgg11_bn",
    "vgg13",
    "vgg13_bn",
    "vgg16",
    "vgg16_bn",
    "vgg19",
    "vgg19_bn",
    "dpn68",
    "dpn98",
    "dpn131",
    "inceptionresnetv2",
    "inceptionv4",
    "xception",
    "densenet161",
    "timm-resnest14d",
    "timm-resnest50d",
    "timm-resnest101e",
    "timm-resnest200e",
    "timm-resnest50d_4s2x40d",
    "timm-res2net50_26w_4s","timm-res2net101_26w_4s","timm-res2net50_26w_6s"
    ]

# encoder_name_list = [str(string).split(".")[-1][:-2] for string in model_list]
for seg_model,seg_model_name in zip(encoder_list,encoder_list):
    def build_model():
        model = smp.UnetPlusPlus(
            encoder_name=seg_model ,     # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
            in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=1   ,     # model output channels (number of classes in your dataset)
            activation=None,
        )
        model.to('cuda')
        return model

    def load_model(path):
        model = build_model()
        model.load_state_dict(torch.load(path))
        model.eval()
        return model

    for fold in range(1):
        print(f'#'*15)
        print(f'### Fold: {fold}')
        print(f'#'*15)
        # run = wandb.init(project='uw-maddison-gi-tract', 
        #                  config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
        #                  anonymous=anonymous,
        #                  name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
        #                  group=CFG.comment,
        #                 )
        train_loader, valid_loader = train_loader,valid_loader
        model     =   build_model()
        # UNet(1,1).to('cuda')
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
        scheduler =  fetch_scheduler(optimizer) 
        # fetch_scheduler(optimizer)
        
        model, history = run_training(model, optimizer, scheduler,
                                    device=CFG.device,
                                    num_epochs=CFG.epochs)
        # run.finish()
        # display(ipd.IFrame(run.url, width=1000, height=720))
        plt.figure(figsize=(12,9))

        plt.plot(history['Train Loss'], label=f'Training Loss')
        plt.plot(history['Valid Loss'], label=f'Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title(f'Loss History-model{seg_model_name}')


        plt.figure(figsize=(17,9))
        plt.subplot(1, 2, 1)
        plt.plot(history['Train Dice'], label=f'Training Dice')
        plt.plot(history['Valid Dice'], label=f'Validation Dice')
        plt.xlabel('Epoch')
        plt.ylabel('Dice')
        plt.legend()
        plt.title(f'Dice History-model{seg_model_name}')

        plt.subplot(1, 2, 2)
        plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
        plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
        plt.xlabel('Epoch')
        plt.ylabel('Jaccard')
        plt.legend()
        plt.title(f'Jaccard History-model{seg_model_name}')
        plt.show()

    history_all_encoder_2[f'{seg_model_name}'].append(history)
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    GPUtil.showUtilization()
    free_memory = gpu.memoryFree
    torch.cuda.empty_cache()

for seg_model_name, history in history_all_encoder_2.items():

        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)
    

In [ ]:
history_all = defaultdict(list)
model_list = [
        smp.Unet,
        smp.UnetPlusPlus,
        smp.MAnet,
        smp.Linknet,
        smp.FPN,
        smp.PSPNet,
        smp.DeepLabV3,
        smp.DeepLabV3Plus,
        smp.PAN
    ]
model_name_list = [str(string).split(".")[-1][:-2] for string in model_list]
for seg_model,seg_model_name in zip(model_list,model_name_list):
    def build_model():
        model = seg_model(
            encoder_name='mobilenet_v2' ,     # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
            in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=1   ,     # model output channels (number of classes in your dataset)
            activation=None,
        )
        model.to('cuda')
        return model

    def load_model(path):
        model = build_model()
        model.load_state_dict(torch.load(path))
        model.eval()
        return model

    for fold in range(1):
        print(f'#'*15)
        print(f'### Fold: {fold}')
        print(f'#'*15)
        # run = wandb.init(project='uw-maddison-gi-tract', 
        #                  config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
        #                  anonymous=anonymous,
        #                  name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
        #                  group=CFG.comment,
        #                 )
        train_loader, valid_loader = train_loader,valid_loader
        model     =   build_model()
        # UNet(1,1).to('cuda')
        optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
        scheduler =  fetch_scheduler(optimizer) 
        # fetch_scheduler(optimizer)
        
        model, history = run_training(model, optimizer, scheduler,
                                    device=CFG.device,
                                    num_epochs=CFG.epochs)
        # run.finish()
        # display(ipd.IFrame(run.url, width=1000, height=720))
        plt.figure(figsize=(12,9))

        plt.plot(history['Train Loss'], label=f'Training Loss')
        plt.plot(history['Valid Loss'], label=f'Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title(f'Loss History-model{seg_model_name}')


        plt.figure(figsize=(17,9))
        plt.subplot(1, 2, 1)
        plt.plot(history['Train Dice'], label=f'Training Dice')
        plt.plot(history['Valid Dice'], label=f'Validation Dice')
        plt.xlabel('Epoch')
        plt.ylabel('Dice')
        plt.legend()
        plt.title(f'Dice History-model{seg_model_name}')

        plt.subplot(1, 2, 2)
        plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
        plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
        plt.xlabel('Epoch')
        plt.ylabel('Jaccard')
        plt.legend()
        plt.title(f'Jaccard History-model{seg_model_name}')
        plt.show()
    history_all[f'{seg_model_name}'].append(history)
    

In [ ]:
plt.figure(figsize=(12,9))

plt.plot(history['Train Loss'], label=f'Training Loss')
plt.plot(history['Valid Loss'], label=f'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title(f'Loss History-model{seg_model_name}')


plt.figure(figsize=(17,9))
plt.subplot(1, 2, 1)
plt.plot(history['Train Dice'], label=f'Training Dice')
plt.plot(history['Valid Dice'], label=f'Validation Dice')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()
plt.title(f'Dice History-model{seg_model_name}')

plt.subplot(1, 2, 2)
plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
plt.xlabel('Epoch')
plt.ylabel('Jaccard')
plt.legend()
plt.title(f'Jaccard History-model{seg_model_name}')
plt.show()

In [ ]:
s = []
for seg_model_name, history in history_all_encoder.items():
    s.append(seg_model_name)
print(s)

In [ ]:
for seg_model_name, history in history_all_encoder.items():

        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)
    

In [ ]:
res = []
res1= defaultdict(list)
for seg_model_name, history in history_all_encoder.items():

        res1['model'].append(seg_model_name+'_e'+str(history[0]['Valid Dice'].index(max(history[0]['Valid Dice']))))
        res1['IOU'].append(max(history[0]['Valid Dice']))
        res.append([f"MODEL: {seg_model_name} VAL IOU MAX:{max(history[0]['Valid Jaccard'])}"])
        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
history_all_encoder_2

In [ ]:
res2 = defaultdict(list)
for seg_model_name, history in history_all_encoder_2.items():
        res2['model'].append(seg_model_name+'_e'+str(history[0]['Valid Jaccard'].index(max(history[0]['Valid Jaccard']))))
        res2['IOU'].append(max(history[0]['Valid Jaccard']))
        # res2[f'{seg_model_name}'].append(max(history[0]['Valid Jaccard']))
        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
res3 = defaultdict(list)
for seg_model_name, history in history_all_sota.items():
        res3['model'].append(seg_model_name+'_e'+str(history[0]['Valid Jaccard'].index(max(history[0]['Valid Jaccard']))))
        res3['IOU'].append(max(history[0]['Valid Jaccard']))
        # res3[f'{seg_model_name}'].append(max(history[0]['Valid Jaccard']))

        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
for seg_model_name, history in history_all_encoder.items():
    if seg_model_name =='resnet50' or seg_model_name =='resnext50_32x4d' :
        plt.figure(figsize=(12,9))

        plt.plot(history[0]['Train Loss'][5:], label=f'Training Loss')
        plt.plot(history[0]['Valid Loss'][5:], label=f'Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title(f'Loss History-model{seg_model_name}')


        plt.figure(figsize=(17,9))
        plt.subplot(1, 2, 1)
        plt.plot(history[0]['Train Dice'], label=f'Training Dice')
        plt.plot(history[0]['Valid Dice'], label=f'Validation Dice')
        plt.xlabel('Epoch')
        plt.ylabel('Dice')
        plt.legend()
        plt.title(f'Dice History-model{seg_model_name}')

        plt.subplot(1, 2, 2)
        plt.plot(history[0]['Train Jaccard'], label=f'Training Jaccard')
        plt.plot(history[0]['Valid Jaccard'], label=f'Validation Jaccard')
        plt.xlabel('Epoch')
        plt.ylabel('Jaccard')
        plt.legend()
        plt.title(f'Jaccard History-model{seg_model_name}')
        plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dictionary
my_dict = {'name1': 10, 'name2': 20, 'name3': 10}

# Extract values
values = list(my_dict.values())

# Create histogram plot
plt.hist(x=df['IOU'])

# Add labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram')

# Show the plot
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
for seg_model_name, history in history_all.items():
    if seg_model_name !='Unet':
        plt.figure(figsize=(12,9))

        plt.plot(history[0]['Train Loss'][2:], label=f'Training Loss')
        plt.plot(history[0]['Valid Loss'][2:], label=f'Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title(f'Loss History-model{seg_model_name}')


        plt.figure(figsize=(17,9))
        plt.subplot(1, 2, 1)
        plt.plot(history[0]['Train Dice'], label=f'Training Dice')
        plt.plot(history[0]['Valid Dice'], label=f'Validation Dice')
        plt.xlabel('Epoch')
        plt.ylabel('Dice')
        plt.legend()
        plt.title(f'Dice History-model{seg_model_name}')

        plt.subplot(1, 2, 2)
        plt.plot(history[0]['Train Jaccard'], label=f'Training Jaccard')
        plt.plot(history[0]['Valid Jaccard'], label=f'Validation Jaccard')
        plt.xlabel('Epoch')
        plt.ylabel('Jaccard')
        plt.legend()
        plt.title(f'Jaccard History-model{seg_model_name}')
        plt.show()

In [ ]:
for seg_model_name, history in history_all.items():
    if seg_model_name !='Unet':
        print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history[0]['Train Dice'])} IOU MAX:{max(history[0]['Train Jaccard'])} Train Loss MIN:{min(history[0]['Train Loss'])} 
        MODEL: {seg_model_name} VAL DICE MAX:{max(history[0]['Valid Dice'])} VAL IOU MAX:{max(history[0]['Valid Jaccard'])} VAL Loss MIN:{min(history[0]['Valid Loss'])} \n """)

In [ ]:
for seg_model_name, history in history_all.items():
    print(seg_model_name, history)

In [ ]:
plt.figure(figsize=(12,9))

plt.plot(history['Train Loss'], label=f'Training Loss')
plt.plot(history['Valid Loss'], label=f'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()


plt.figure(figsize=(17,9))
plt.subplot(1, 2, 1)
plt.plot(history['Train Dice'], label=f'Training Dice')
plt.plot(history['Valid Dice'], label=f'Validation Dice')
plt.xlabel('Epoch')
plt.ylabel('Dice')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['Train Jaccard'], label=f'Training Jaccard')
plt.plot(history['Valid Jaccard'], label=f'Validation Jaccard')
plt.xlabel('Epoch')
plt.ylabel('Jaccard')
plt.legend()

In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('resnet18', pretrained=None)
print(preprocess_input)

In [ ]:
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [ ]:
weight_c1 = 'UNetPPEffb6_600_std.pth'
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b6", encoder_weights=None, in_channels=1, classes=1)
# weight_c1 = path2 / 'deeplabv3plus.pkl'
# self._model = smp.DeepLabV3Plus(
#     encoder_name="efficientnet-b6", encoder_weights=None, in_channels=1, classes=1)
model.to('cuda')
model.load_state_dict(torch.load(
    weight_c1, map_location='cuda'))
model.eval()


In [ ]:
target_shape

In [ ]:
valid_dataset = BuildDataset_prod(x_val, transforms=transforms_prod['prod'])

In [ ]:

imgs= next(iter(valid_dataset))
# imgs = imgs.to('cuda')
val_scores=[]
preds = []
print(sum(imgs))
# imgs = imgs[None, :,:,:]
# mask = mask[None, :,:,:]
for ings in valid_loader:
    for fold in range(1):
        # model = load_model(f"best_epoch-{fold:02d}.bin")
        with torch.no_grad():
            pred = model(imgs).cpu()
            pred = (nn.Sigmoid()(pred)>0.5).double()
            
            # preds = torch.argmax(nn.Sigmoid()(pred),axis=1).double()
            # val_dice = dice_coef(mask, pred).cpu().detach().numpy()
            # val_jaccard = iou_coef(mask, pred).cpu().detach().numpy()
            # val_scores.append([val_dice, val_jaccard])
            
            
            print( sum(sum(sum(sum(pred)))))
        preds.append(pred)
        
    imgs  = imgs.cpu().detach()
    preds = torch.mean(torch.stack(preds, dim=0), dim=0).cpu().detach()



In [ ]:
device='cpu'

In [ ]:
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [ ]:
preds = []
for step, (images, masks) in enumerate(valid_loader):        
        images  = images.to(device, dtype=torch.float).cpu()
        masks   = masks.to(device, dtype=torch.float).cpu()
        models=model.cpu()

        for fold in range(1):
            # model = load_model(f"best_epoch-{fold:02d}.bin")
            with torch.no_grad():
                pred = models(images).cpu()
                pred = (nn.Sigmoid()(pred)>0.5).double()
                
                # preds = torch.argmax(nn.Sigmoid()(pred),axis=1).double()
                # val_dice = dice_coef(mask, pred).cpu().detach().numpy()
                # val_jaccard = iou_coef(mask, pred).cpu().detach().numpy()
                # val_scores.append([val_dice, val_jaccard])
                
                
                print( sum(sum(sum(sum(pred)))))
            preds.append(pred)
            
        images  = images.cpu().detach()
        preds = torch.mean(torch.stack(preds, dim=0), dim=0).cpu().detach()
        break


In [ ]:
plot_batch(pred, mask, size=1)

In [ ]:
def dice_coef_2(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    smooth = 0.0001
    intersection = (y_true*y_pred).sum()
    print(intersection)
    print(((y_true_f).sum() + (y_pred_f).sum() + smooth))
    print((y_true_f).sum())
    print((y_pred_f).sum())
    return (2. * intersection + smooth) / ((y_true_f).sum() + (y_pred_f).sum() + smooth)

In [ ]:

def dice_coef(y_true, y_pred, thr=0.3, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

In [ ]:
dice_coef(masks, preds).cpu().detach().numpy()

In [ ]:
iou_coef(masks, preds).cpu().detach().numpy()

In [ ]:
print(val_scores)

In [ ]:
torch.save(model.s, 'UNetPPEffb6_600_2.pt')

In [ ]:
torch.save(model.state_dict(), 'UNetPPEffb6_600_std.pth')

In [ ]:
sum(sum(sum(mask[:1])))

In [ ]:
plot_batch(images, images, size=5)
plot_batch(images, masks, size=5)
plot_batch(images, preds, size=5)
